In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
print(matplotlib.get_backend())


module://ipykernel.pylab.backend_inline


#  寶可夢自動編碼器 (tensorflow)

支援python 版本: 3.5以上  
支援pytorch版本 : 1.2以上

深度學習的關鍵就是「表徵學習( representation learning)」，透過最佳化算法算更新神經元權重的同時，也正是將所搜尋到的特徵進一步檢視是有用的保留，還是冗餘的捨棄，而  autoencoder  自動編碼器正是找尋關鍵特徵並將其充分壓縮的經典網路結構。在這個實作範例中，我們將帶著大家設計一個簡單的卷積自編碼器，而輸入的數據正是目前很流行的寶可夢，我們要來實證看看，光是利用沒有做任何標註的數據，自編碼器是否能夠有效的找出關鍵特徵。

In [3]:
import glob
import os
import cv2
os.environ['TRIDENT_BACKEND'] = 'pytorch'
os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'
!pip uninstall tridentx
!pip install '/content/gdrive/My Drive/DeepBelief_Course5_Examples/tridentx-0.5.0-py3-none-any.whl'
#!pip install tridentx --upgrade
import trident as T
from trident import *

Processing ./gdrive/My Drive/DeepBelief_Course5_Examples/tridentx-0.5.0-py3-none-any.whl


trident 0.5.0
Using Pytorch backend.
Image Data Format: channels_first.
Image Channel Order: rgb.
Using pillow image backend.
Pillow version:7.0.0.
Pytorch version:1.4.0.


首先載入這次這個實作所需要的工具包，各位可以發現我沒有匯入pytorch相關的包，而改匯入trident ， trident 是我為了便利教學以及簡化開發流程所設計的新的 api ，開發它的原因在於不管是不熟悉 python的初學者或是精通深度學習的開發者。都會因為建立分析過程中各種繁瑣的細節、框架的差異以及常疏漏的設定等而陷入在痛苦的填坑之路， trident 希望學習者或開發者可以不用再重複造輪子以及希望能降低大家掉到坑裡的機率。各位可以直接利用 pip install tridentx來安裝(如果是在 jupyter notebook上執行安裝請記得前方加 !)

trident能夠如何簡化分析流程呢?以下一句語法為例，我們只需要一行語法就能直接下載並且讀取我們課程的數據集( 除了上課範例數據集外， trident也內建了不少經典數據集)，各位可能覺得那有甚麼稀奇， keras以及 torchvision早就有了這功能， trident跟他們不同的地方，若是您宣告了

os.environ['TRIDENT_BACKEND'] = 'pytorch'

那麼， trident 就會自動地把影像數據格式轉成以 CHW, BGR, 目標標籤不做 onehot ，若是 tensorflow自動轉成 HWC, RGB, 目標標籤做 onehot ，若是 cntk 自動轉成 CHW, BGR, 目標標籤做 onehot ，若是使用  opencv 則會自動將 BGR轉 RGB，覺得有點意思了吧。 trident 這個字的原意是三叉戟，代表著這三個深度學習框架，我並非要像 keras一樣設計一個高階 api企圖一統各種框架，但實際上卻因為遷就框架之間差異太大失去了原有的簡潔性，相反的我就是理解以及知道框架之間的差異過大，因此 trident 比較像是一個跨框架的平行開發範本，我們盡量在不同框架中能讓各位有一致的開發體驗，也就是學一次就能輕易地跨框架使用。

In [4]:
dataset=T.load_examples_data('pokemon')

archive file is already existing, donnot need download again.
extraction is finished, donnot need extract again.
/content/gdrive/My Drive/trident/datasets/examples_pokemon/pokemon
get pokemon images :1444


覺得torchvision 的 transform 開發很不方便嗎，在 trident 所有的轉換都是基於函數，輸入是圖檔(看影像後台是用什麼，預設是pillow)，輸出則是處理過的影像圖檔，至於執行的順序只需要用清單依序放置，指派給資料集的image_transform_funcs就可以了 。例如下面的語法就是先將影像大小縮放至 128*128 ，在正規化(減 127.5 除以127.5)，這樣數據在吐出來之前就會依照這些轉換依序完成。

In [0]:
dataset.image_transform_funcs=[resize((130,130)),
                               random_crop(128,128),
                               normalize(127.5,127.5)]       

下圖是標準的自動編碼器的結構示意圖，它透過前半段尺寸越來越小的編碼器，將原始圖片壓縮編碼成特徵向量(在本次實作中是長度為 128的向量)後，在透過後半段尺寸逐漸增大的解碼器進行結構上的復原，而損失函數一般使用 MSELoss 也就是輸入圖與輸出圖差異的平方，目的就是希望神經網路夠找到一個最佳的編碼方式，讓極度壓縮過後的訊息能夠重建回原狀。

<img src='https://docs.google.com/uc?export=download&id=1-brTOHQ7oh8Y24u-CQoYn1uq4VE9P_Fj' width='600px'/>

以下是我們分別設計模型的編碼器與解碼器部分，模型設計也是 trident比較特殊之處，因為在原本的 torch 中所有的卷積層都需要指定輸入的通道數( input-channel) ，同時也需要自己指定 padding 需要的量，同時一般建模所常用的卷積 ->正規化 ->活化函數都需要自己一層一層指定或者是自己把它包成自訂模組，在 trident 中不需要層層指定輸入通道數，只需要第一層指定輸入形狀、或是第一次 forward自行偵測輸入形狀、也可以透過shape_infer([input_size]) 就可以自動更新全體的input_filters ，此外padding 也可以替換為 auto_pad 即可自動計算保持原形狀所需的 padding量，同時內建的Conv2d_Block直接讓卷積、正規化、活化函數、 dropout 和添加噪音等功能都集合在一起，這樣開發起來就更方便了。

In [0]:

encoder=Sequential(
    Conv2d_Block((5,5),32,strides=1,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False, add_noise=True,noise_intensity=0.05),#(32,128,128)
    Conv2d_Block((3,3),64,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(64,64,64)
    Conv2d_Block((3,3),64,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(64,32,32)
    Conv2d_Block((3,3),128,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False,dropout_rate=0.5),#(128,16,16)
    Conv2d_Block((3,3),128,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(128,8,8)
    Conv2d_Block((3,3),256,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(256,4,4)
    Conv2d_Block((3,3),256,strides=1,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(256,4,4)
    Reshape((-1,1,1)), #(256*4*4)
    Conv2d((1,1),128,strides=1,auto_pad=True,activation='tanh',use_bias=False)
)


decoder=Sequential(
    Conv2d((1,1),128*4*4,strides=1,auto_pad=True,activation='tanh',use_bias=False), #(2048,1,1 )
    Reshape((128,4,4)), #(128,4,4))
    Conv2d_Block((3,3),128,strides=1,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False) ,#((128,4,4))
    TransConv2d_Block((3,3),128,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(128,8,8)
    TransConv2d_Block((3,3),64,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(64,16,16)
    TransConv2d_Block((3,3),64,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(64,32,32)
    TransConv2d_Block((3,3),64,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(64,64,64)
    TransConv2d_Block((3,3),64,strides=2,auto_pad=True,activation='leaky_relu',normalization='batch',use_bias=False),#(64,128,128)
    Conv2d((1,1),3,strides=1,auto_pad=True,activation='tanh',use_bias=False)
)




自動編碼器的模型就只要將編碼器與解碼器連在一起就完成了

In [0]:
autoencoder=Sequential(
    encoder,
    decoder
)

剛才提到了有關於輸入形狀的推斷， trident 中的所有神經網路層都是支援延遲推斷的，這樣的好處是在於不需要逐層設定輸入通道數，只需要做一次(整體模型，也就是最外層)即可



In [12]:
autoencoder.input_shape=[3,128,128]
summary(autoencoder,(3,128,128))
#如果你不想重頭跑，就請把下一行的註釋取消
#autoencoder=T.load('Models/pokemon_ae.pth')

--------------------------------------------------------------------------------------------------------------------------------
              Layer (type)                   Output Shape            Weight           Bias    Param #     FLOPS #   
conv2d_28                                [-1, 32, 128, 128]    [32, 3, 5, 5]                  2400      78626816.0  
batch_norm_14                            [-1, 32, 128, 128]    [32]                 [32]      64        1556480.0   
conv2d__block_9                          [-1, 32, 128, 128]                                   0         0.0         
batch_norm_15                            [-1, 64, 64, 64]      [64]                 [64]      128       782336.0    
conv2d__block_10                         [-1, 64, 64, 64]                                     0         0.0         
batch_norm_16                            [-1, 64, 32, 32]      [64]                 [64]      128       195584.0    
conv2d__block_11                         [-1, 64, 32

傳統的pytorch訓練時非常繁瑣，像是要切換訓練模式model.train()、清掉梯度zero_grad()、反向傳播 loss.backward() 、進行下個批次 optimizer.step() ，這些已經夠煩人了，更別提向量進入模型前得轉 tensor ，要轉回 numpy 還得 detach().cpu().numpy()[0] ，還有無處不在的 cpu(),cuda()，是不是許多執行失敗都是出在這些瑣碎小細節遺漏了呢? trident 中我重新包裝了 TrainingPlan的容器，在其中已經將訓練過程的這些細節通通都封裝好了，甚至包括多久列印出來進度、多久顯示損失函數變化曲線、多久存檔一次 ... 這些通通都可以很直覺化的設定，而且使用了Fluent Code風格，讓它可以很方便設定以及增加了可讀性與維護性。

In [13]:
model=Model(input_shape=[3,128,128],output=autoencoder)\
    .with_optimizer(optimizer='Ranger',lr=2e-3,betas=(0.9, 0.999))\
    .with_loss(MSELoss,loss_weight=1,name='l2 loss')\
    .with_loss(EdgeLoss,loss_weight=0.2,name='edge loss')\
    .with_metric(rmse,name='rmse')\
    .with_regularizer('l2')\
    .with_constraint('max_min_norm')\
    .with_learning_rate_scheduler(reduce_lr_on_plateau,monitor='rmse',mode='min',factor=0.5,patience=5,cooldown=1,threshold=5e-4,warmup=5)\
    .with_model_save_path('/content/gdrive/My Drive/DeepBelief_Course5_Examples/prewarm03_自動寶可夢編碼器/Models/pokemon_ae_tf.pth')\
    .with_callbacks(TileImageCallback(batch_inteval=10,name_prefix= 'srresnet_v2_tile_image_{0}.png',include_input=True,include_output=True,include_target=False,imshow=True,
                                      save_path='/content/gdrive/My Drive/DeepBelief_Course5_Examples/prewarm03_自動寶可夢編碼器/results/'))

plan=TrainingPlan()\
    .add_training_item(model)\
    .with_data_loader(dataset)\
    .repeat_epochs(250)\
    .within_minibatch_size(32)\
    .print_progress_scheduling(20,unit='batch')\
    .save_model_scheduling(10,unit='batch')
 


l2 loss signature:[('output', [3, 128, 128]), ('target', [3, 128, 128])]
edge loss signature:[('output', [3, 128, 128]), ('target', [3, 128, 128])]
rmse signature:[('output', [3, 128, 128]), ('target', [3, 128, 128])]


所以以上TrainingPlan的設定就是

加入TrainingItem (包含了模型、優化器、損失函數、效度指標、權重正則)
指定 data_loader
重複250 epoch
指定大小
指定學習率變化模式
印出學習進度
指定模型存檔週期
顯示autoencoder 效果輸出圖
指定繪製損失函數與指標變動歷史圖的週期
除了預設的功能外，事實上也可以利用 Callbacks 的機制在關鍵時間點插入自定義工作，這些特性之後也會在介紹，設定完成後，只需要透過start_now()函數即可啟動。

In [0]:
plan.start_now()

以下是我跑了 500 epoch 的成果，是不是可以看到從雜訊到模糊，一直到越來越清晰的過程呢，只要訓練的時間夠久就能夠越來越清晰。如果您手邊沒有 gpu ，建議可以透過有免費gpu的google 的colab 來執行。

<img src='https://docs.google.com/uc?export=download&id=1U6KEW9eanMO5wytgnMp_SZlM8xRTPGfu' width='600px'/>

## 表徵學習 

可能會有人覺得奇怪，自編碼器重建圖像就算訓練好了，到底是有甚麼作用？其實，對我們有用的並不是整個自編碼器，我們要的其實是前半段的編碼器部分。編碼器的工作是將圖片編碼成長度為 128 向量(為了避免使用到全連接層，所以實際上是 (128,1,1) 的形狀)，等於是將圖片抽出它的關鍵特徵，而這些特徵既然可以用來還原回圖像細節，這表示它必定包含了這個圖片中的關鍵訊息。這也是深度學習中表徵學習(representation learning)中最常見的手法。而這些特徵向量就可以幫助我們評估圖片中的相似性，也就是可以做到視覺搜索的效果。

我們首先利用資料源的get_all_data()函數取出所有圖片，依序透過編碼器(autoencoder[0])產生特徵向量，並將1444 張圖片的特徵向量整併成尺寸為 (1444,128) 的向量

In [0]:
features=[]


#dataset.data['train]
for img_data in dataset.get_all_data():
    input=to_tensor(np.expand_dims(dataset.image_transform(img_data),0))
    encoder_output = np.squeeze(to_numpy(autoencoder[0](input)))
    features.append(encoder_output)


features=np.asarray(features)
print(features.shape)
   

我們如果想要看一下特徵向量整體的效果可透過傳說中的降維神器 t-SNE ，將特徵向量降為並且視覺化。

In [0]:
from matplotlib import offsetbox
from sklearn import manifold
import PIL
from PIL import Image as Image

def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    fig =plt.figure(figsize=(18,18))
    ax = plt.subplot(111)

    if hasattr(offsetbox, 'AnnotationBbox'):
        # 需要matplotlib 版本> 1.0才支援顯示圖片功能
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
           
            shown_images = np.r_[shown_images, [X[i]]]
            #將向量轉圖片，且將圖片縮小至32*32
            img=array2image(dataset.data['train'][i]) #array2image是 trident 內的函數
            img = img.resize((32, 32),Image.ANTIALIAS)
            imagebox = offsetbox.AnnotationBbox(offsetbox.OffsetImage(img),X[i],pad=0, box_alignment=(0, 0))
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
    display.display(fig)

#利用 t-SNE  降維
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)#利用t-sne將128特徵向量降維至2
X_tsne = tsne.fit_transform(features[:500,:])#為了避免圖片太密我只用了前500來處理

plot_embedding(X_tsne, "Embedded of Pokemon")#繪製圖像散布圖

看起來頗壯觀，仔細看的確位置比較接近的都有向是姿態、顏色、類型的相似，那麼我們該如何達到視覺搜索的效果呢?

## 寶可夢的相似性搜索 

特徵向量因為在學習過程中被多次正規化，因此計算歐幾里得距離是沒有意義的，所以一般是用 cosine距離，在這種逐一比較 cosine距離的場景下，若是一個一個比實在太累了， trident 內建了一次產生各成員 cosine距離的函數element_cosine_distance，所以可以一次性的比較，然後取出距離最大者( cosine距離跟其他距離不一樣，越相似者 cosine距離越大)。



In [0]:
def find_similar_pokemon(idx):
    similarity_list=[]
    result=to_numpy(element_cosine_distance(features[idx:idx+1,:],features))  #element_cosine_distance逐成員計算Cosine距離


    top5=np.argsort(result)[-5:][::-1]  #找出前 5個Cosine距離最高者(Cosine距離是越高越像)
    similarity_list=[dataset.data['train'][idx]] #放入原圖
    similarity_list.append(np.ones_like(similarity_list[0])[:,:30,:]*255) # 加入白色分隔線
    similarity_list.extend(dataset.data['train'][top5]) #放入前 5名圖 

    merge_img=np.concatenate(similarity_list,axis=1) #沿著寬(axis=1)疊合
    display.display(array2image(merge_img)) #顯示結果



idx=128 #抽取一隻寶可夢
find_similar_pokemon(idx)

我們當然也可以把傳入的索引值透過隨機的方式來指派，這樣我們就可以觀察到更多樣的搜索結果

In [0]:
import random
find_similar_pokemon(random.choice(range(1444)))